<a href="https://colab.research.google.com/github/EnzoGui18/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q \
    transformers \
    torch \
    accelerate \
    bitsandbytes \
    sentence-transformers \
    faiss-cpu \
    gradio \
    pdfplumber \
    beautifulsoup4 \
    requests \
    langchain \
    langchain-community \
    langchain-text-splitters

print("✅ Ambiente completo para execução local no Colab instalado!")

✅ Ambiente completo para execução local no Colab instalado!


In [ ]:
from huggingface_hub import login

# Cole seu token de acesso do Hugging Face
login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer

# Define o dispositivo como a GPU do Colab
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

# --- 1. Carregar o Modelo de Embedding ---
print("\nCarregando modelo de embedding 'Legal-BERTimbau-large'...")
embedding_model = SentenceTransformer('rufimelo/Legal-BERTimbau-large', device=device)
print("✅ Modelo de Embedding carregado.")

# --- 2. Carregar o Modelo de Linguagem (LLM) com Quantização ---
LLM_MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"

# Configuração de quantização correta
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"\nCarregando LLM '{LLM_MODEL_NAME}' (pode demorar)...")
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME)
llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)
print("✅ LLM carregado com sucesso!")

Usando dispositivo: cuda

Carregando modelo de embedding 'Legal-BERTimbau-large'...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at rufimelo/Legal-BERTimbau-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modelo de Embedding carregado.

Carregando LLM 'meta-llama/Meta-Llama-3-8B-Instruct' (pode demorar)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ LLM carregado com sucesso!


In [ ]:
from pathlib import Path
from typing import List, Dict
import pdfplumber
import requests
from bs4 import BeautifulSoup

class PDFReader:
    def __init__(self, data_dir: str = "data"):
        self.data_dir = Path(data_dir)
        self._create_directory()
    def _create_directory(self) -> None:
        if not self.data_dir.exists(): self.data_dir.mkdir(parents=True, exist_ok=True)
    def _find_pdfs(self) -> List[Path]:
        return list(self.data_dir.glob("*.pdf"))
    def _extract_text(self, pdf_path: Path) -> str:
        text = ""
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    page_text = page.extract_text(); text += page_text + "\n" if page_text else ""
        except Exception as e: print(f"❌ Erro ao processar {pdf_path.name}: {e}")
        return text
    def read_all_pdfs(self) -> Dict[str, str]:
        pdf_files = self._find_pdfs()
        if not pdf_files: print(f"❌ Nenhum PDF encontrado em '{self.data_dir}'"); return {}
        documents = {}; print(f"🔎 Processando {len(pdf_files)} arquivo(s) PDF...")
        for pdf_path in pdf_files:
            text = self._extract_text(pdf_path)
            if text.strip(): documents[pdf_path.name] = text; print(f"📄✔ {pdf_path.name}")
        return documents

class DocumentLoader:
    def __init__(self, data_dir: str = "data"):
        self.pdf_reader = PDFReader(data_dir); self.documents = {}
    def load_pdfs(self):
        print("\n📄 Carregando PDFs..."); pdf_docs = self.pdf_reader.read_all_pdfs(); self.documents.update(pdf_docs)
    def load_all(self):
        self.load_pdfs()
        print("\n" + "="*30 + "\n📊 Resumo do Carregamento:")
        print(f"  - Total de documentos: {len(self.documents)}")
        print(f"  - Total de caracteres: {sum(len(c) for c in self.documents.values())}")
        print("="*30)

print("✅ Classes PDFReader e DocumentLoader definidas.")

✅ Classes PDFReader e DocumentLoader definidas.


In [ ]:
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter

def clean_text(text: str) -> str:
    text = re.sub(r'\n{3,}', '\n\n', text); text = re.sub(r'\s{2,}', ' ', text)
    lines = text.split('\n'); clean_lines = [line.strip() for line in lines if len(line.strip()) > 10]
    return "\n".join(clean_lines)

def create_chunks(documents: Dict[str, str], chunk_size=1000, chunk_overlap=200) -> Dict[str, List[str]]:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunked_docs = {}; print("\n쪼 Dividindo documentos em chunks...")
    for doc_name, content in documents.items():
        chunks = text_splitter.split_text(content)
        chunked_docs[doc_name] = chunks
        print(f"  - '{doc_name}' dividido em {len(chunks)} chunks.")
    return chunked_docs

print("✅ Funções clean_text e create_chunks definidas.")

✅ Funções clean_text e create_chunks definidas.


In [ ]:
# 1. Carregar os documentos
loader = DocumentLoader()
loader.load_all()
raw_documents = loader.documents

# 2. Limpar os textos
cleaned_documents = {doc_name: clean_text(content) for doc_name, content in raw_documents.items()}
print("\n✨ Textos limpos com sucesso!")

# 3. Dividir em Chunks
chunked_documents = create_chunks(cleaned_documents)
print("\n✅ Pipeline de preparação de dados concluído!")


📄 Carregando PDFs...
🔎 Processando 3 arquivo(s) PDF...
📄✔ Consumidor.pdf
📄✔ civil.pdf
📄✔ trabalhista.pdf

📊 Resumo do Carregamento:
  - Total de documentos: 3
  - Total de caracteres: 2125765

✨ Textos limpos com sucesso!

쪼 Dividindo documentos em chunks...
  - 'Consumidor.pdf' dividido em 104 chunks.
  - 'civil.pdf' dividido em 822 chunks.
  - 'trabalhista.pdf' dividido em 1721 chunks.

✅ Pipeline de preparação de dados concluído!


In [ ]:
# Célula 8 (Corrigida e Completa)

from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings # <-- IMPORTANTE: A importação do nosso "adaptador"

# O FAISS prefere uma lista de objetos 'Document' do LangChain.
# Esta parte do código já está correta.
documents_for_faiss = []
for doc_name, chunks in chunked_documents.items():
    for chunk_text in chunks:
        documents_for_faiss.append(
            Document(page_content=chunk_text, metadata={"source": doc_name})
        )

print(f"\nTotal de {len(documents_for_faiss)} chunks preparados para vetorização.")

# --- A CORREÇÃO ESTÁ AQUI ---
# 1. Criamos o "adaptador" compatível com o LangChain.
# Em vez de passar nosso 'embedding_model' diretamente, nós instanciamos
# a classe HuggingFaceEmbeddings, especificando o nome do modelo.
langchain_embedder = HuggingFaceEmbeddings(
    model_name='rufimelo/Legal-BERTimbau-large',
    model_kwargs={'device': device} # Garante que ele use a GPU que já definimos
)
print("Adaptador de embedding para LangChain criado.")

print("\nVetorizando os documentos e criando o índice FAISS...")

# 2. Usamos o novo 'langchain_embedder' para criar o vector_store
vector_store = FAISS.from_documents(documents_for_faiss, langchain_embedder)

print("✅ Banco de dados vetorial FAISS criado com sucesso!")


Total de 2647 chunks preparados para vetorização.


/tmp/ipython-input-715655568.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  langchain_embedder = HuggingFaceEmbeddings(
Some weights of BertModel were not initialized from the model checkpoint at rufimelo/Legal-BERTimbau-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Adaptador de embedding para LangChain criado.

Vetorizando os documentos e criando o índice FAISS...
✅ Banco de dados vetorial FAISS criado com sucesso!


In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

print("Preparando o LLM para integração com o LangChain...")

# Cria um pipeline de geração de texto do Transformers
# Este é o invólucro padrão para usar modelos como o Llama 3
text_generation_pipeline = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    max_new_tokens=1024, # Define o tamanho máximo da resposta
    torch_dtype=torch.bfloat16
)

# "Embrulha" o pipeline na classe compatível com LangChain
llm_for_chain = HuggingFacePipeline(pipeline=text_generation_pipeline)

print("✅ LLM pronto para ser usado na cadeia RAG!")

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


Preparando o LLM para integração com o LangChain...
✅ LLM pronto para ser usado na cadeia RAG!


/tmp/ipython-input-3970723560.py:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm_for_chain = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("Construindo a cadeia RAG...")

# --- 1. Definir o Retriever a partir do nosso Vector Store ---

retriever = vector_store.as_retriever(search_kwargs={"k": 5}) # k=3 significa que buscará os 3 chunks mais relevantes
# --- 2. Criar o Template do Prompt ---

template = """
Você é um assistente jurídico no Brasil. Use APENAS o contexto a seguir para responder à pergunta.
Se a resposta não estiver no contexto, diga "Com base nos documentos fornecidos, não encontrei a informação."
Cite a fonte do documento de onde extraiu a resposta.

Contexto: {context}

Pergunta: {question}

---
Após a resposta, adicione em uma nova linha sua confiança na resposta de 0 a 100% no formato "Confiança: XX%". A confiança deve ser alta se o contexto responde diretamente à pergunta.

Resposta:
"""
prompt = PromptTemplate.from_template(template)

# --- 3. Função para formatar os documentos recuperados ---
def format_docs(docs):
    return "\n\n".join(f"Fonte: {doc.metadata.get('source', 'N/A')}\n{doc.page_content}" for doc in docs)

# --- 4. Montar a Cadeia (Chain) ---
# Esta é a sequência de passos que o sistema executará
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_for_chain
    | StrOutputParser()
)

print("✅ Cadeia RAG montada e pronta para uso!")

Construindo a cadeia RAG...
✅ Cadeia RAG montada e pronta para uso!


In [ ]:
# Faça uma pergunta relacionada aos PDFs que você subiu
query = "De acordo com o Código de Defesa do Consumidor, qual é o prazo para reclamar de vícios aparentes em produtos não duráveis e em produtos duráveis?"

print(f"Enviando a pergunta: '{query}'")
print("\nAguardando a resposta do sistema RAG...")
print("="*30)

# O método .invoke() executa a cadeia completa
response = rag_chain.invoke(query)

# Imprime a resposta final
print(response)
print("="*30)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Enviando a pergunta: 'De acordo com o Código de Defesa do Consumidor, qual é o prazo para reclamar de vícios aparentes em produtos não duráveis e em produtos duráveis?'

Aguardando a resposta do sistema RAG...

Você é um assistente jurídico no Brasil. Use APENAS o contexto a seguir para responder à pergunta.
Se a resposta não estiver no contexto, diga "Com base nos documentos fornecidos, não encontrei a informação."
Cite a fonte do documento se possível.

Contexto: Fonte: Consumidor.pdf
§ 3° O fornecedor de serviços só não será responsabilizado quando provar:
I - que, tendo prestado o serviço, o defeito inexiste;
II - a culpa exclusiva do consumidor ou de terceiro.
§ 4° A responsabilidade pessoal dos profissionais liberais será apurada mediante a verificação de culpa.
Art. 15. (Vetado).
Art. 16. (Vetado).
Art. 17. Para os efeitos desta Seção, equiparam-se aos consumidores todas as vítimas do evento.
Da Responsabilidade por Vício do Produto e do Serviço
Art. 18. Os fornecedores de produ

In [ ]:
print("Definindo a função de classificação...")

# Prompt para instruir o LLM a agir como um classificador
CLASSIFIER_PROMPT = """
Sua única tarefa é classificar a pergunta do usuário em uma das 4 categorias abaixo. Retorne APENAS o nome da categoria.

Categorias:
1.  'rag_required': A pergunta exige conhecimento de leis, artigos, códigos ou jurisprudência.
2.  'calculation_required': A pergunta pede um cálculo numérico baseado em regras jurídicas.
3.  'general_conversation': É uma saudação, agradecimento ou conversa casual.
4.  'out_of_scope': A pergunta não tem relação com o domínio jurídico.

Pergunta do Usuário: "{user_query}"

Categoria:
"""

# Criamos uma cadeia específica para a classificação
classifier_chain = (
    PromptTemplate.from_template(CLASSIFIER_PROMPT)
    | llm_for_chain # Usamos o mesmo LLM já preparado
    | StrOutputParser()
)

def classify_query(user_query: str) -> str:
    """Classifica a query do usuário em uma das 4 categorias."""
    response = classifier_chain.invoke({"user_query": user_query})
    # Limpeza para garantir que apenas o nome da categoria seja retornado
    clean_response = response.strip().lower()

    valid_categories = ["rag_required", "calculation_required", "general_conversation", "out_of_scope"]
    for category in valid_categories:
        if category in clean_response:
            print(f"Query classificada como: '{category}'")
            return category

    print("Classificação não foi clara, retornando 'out_of_scope'.")
    return "out_of_scope" # Fallback seguro

print("✅ Função 'classify_query' pronta.")

# Teste rápido (opcional)
# classify_query("Olá, tudo bem?")
# classify_query("Como calculo minhas férias?")

Definindo a função de classificação...
✅ Função 'classify_query' pronta.


In [ ]:
import gradio as gr

# --- Disclaimers (Avisos Legais) ---
DISCLAIMER_RAG = "\n\n---\n*AVISO: Esta resposta foi gerada por IA e não substitui a consulta a um profissional do Direito. Verifique as fontes.*"
DISCLAIMER_GENERAL = "\n\n---\n*AVISO: Esta é uma resposta conversacional e não deve ser interpretada como aconselhamento jurídico.*"

# --- A Função Mestra ---
def chatbot_response(message, history):
    """
    Função principal que o Gradio irá chamar.
    Orquestra a classificação e a geração da resposta.
    """
    # 1. Classifica a pergunta
    classification = classify_query(message)

    # 2. Age com base na classificação
    if classification == 'rag_required' or classification == 'calculation_required':
        # Se for RAG, chama a cadeia RAG que já testamos
        response = rag_chain.invoke(message)
        final_response = response.strip() + DISCLAIMER_RAG

    elif classification == 'general_conversation':
        # Se for conversa, usa o LLM com um prompt simples
        general_prompt = f"Você é um assistente virtual. Responda à seguinte mensagem de forma breve e amigável: '{message}'"
        response = llm_for_chain.invoke(general_prompt)
        final_response = response.strip() + DISCLAIMER_GENERAL

    else: # out_of_scope
        # Se for fora de escopo, retorna uma mensagem padrão
        final_response = "Desculpe, meu conhecimento é focado em legislação brasileira. Não consigo ajudar com este tópico."

    return final_response

# --- 3. Inicia a Interface Gradio ---
print("\nIniciando a interface do Chatbot Jurídico...")

gr.ChatInterface(
    fn=chatbot_response,
    title="⚖️ Chatbot Jurídico Inteligente (Local)",
    description="Faça uma pergunta sobre a CLT, Código de Defesa do Consumidor ou Código Civil com base nos arquivos carregados.",
    examples=[
        "Qual o prazo para reclamar de vícios em produtos duráveis?",
        "Como funciona a jornada de trabalho de 12 por 36 horas?",
        "O que é considerado união estável?",
        "Olá, qual seu nome?"
    ],
    cache_examples=False
).launch(debug=True)


Iniciando a interface do Chatbot Jurídico...


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://db7804d9983044d510.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query classificada como: 'rag_required'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query classificada como: 'rag_required'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query classificada como: 'rag_required'
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://db7804d9983044d510.gradio.live


In [ ]:
from sklearn.metrics import classification_report

print("📊 Iniciando a avaliação do classificador...")

# Amostra do Dataset de Teste de Classificação
classification_test_set = {
    "rag_required": [
        "o que a clt fala sobre férias?",
        "qual a punição para o crime de roubo segundo o código penal?",
        "tenho direito a danos morais por cobrança indevida?",
        "como funciona a usucapião de um imóvel?",
    ],
    "calculation_required": [
        "como calculo minha rescisão com aviso prévio?",
        "qual o valor da multa de 40% do FGTS para um saldo de R$ 12.000?",
        "qual o valor de 1/3 de férias sobre um salário de 3000 reais?",
    ],
    "general_conversation": [
        "oi tudo bem?",
        "muito obrigado pela ajuda",
        "quem te criou?",
        "você é um robô?",
    ],
    "out_of_scope": [
        "qual a previsão do tempo para amanhã?",
        "me conte uma piada",
        "qual o melhor time de futebol do brasil?",
        "receita de bolo de cenoura",
    ]
}

true_labels = []
predicted_labels = []

# Executa a classificação para cada item do dataset
for label, queries in classification_test_set.items():
    for query in queries:
        true_labels.append(label)
        predicted_category = classify_query(query)
        predicted_labels.append(predicted_category)

# Gera o relatório de classificação
print("\n" + "="*50)
print("Relatório de Performance do Classificador")
print("="*50)
report = classification_report(true_labels, predicted_labels)
print(report)
print("="*50)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


📊 Iniciando a avaliação do classificador...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query classificada como: 'rag_required'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query classificada como: 'rag_required'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query classificada como: 'rag_required'


KeyboardInterrupt: 

In [ ]:
!pip install -q nltk

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

print("\n\n📊 Iniciando a avaliação do Sistema RAG...")

# Amostra do Dataset de Teste RAG com perguntas e respostas de referência
rag_test_set = [
    {
        "question": "Qual o prazo para reclamar de vícios em produtos duráveis?",
        "reference_answer": "O direito de reclamar por vícios aparentes em produtos duráveis caduca em noventa dias."
    },
    {
        "question": "Até quantas horas extras por dia um funcionário pode fazer segundo a CLT?",
        "reference_answer": "A duração diária do trabalho poderá ser acrescida de horas extras, em número não excedente de duas."
    },
    {
        "question": "O que acontece se eu me demitir sem justa causa antes do fim do contrato com prazo determinado?",
        "reference_answer": "Havendo termo estipulado, o empregado não se poderá desligar do contrato, sem justa causa, sob pena de ser obrigado a indenizar o empregador dos prejuízos que desse fato lhe resultarem."
    }
]

total_bleu_score = 0

print("\n" + "="*50)
print("Avaliando Respostas do RAG")
print("="*50)

for item in rag_test_set:
    question = item["question"]
    reference = item["reference_answer"].split() # A resposta de referência como lista de palavras

    # Gera a resposta com nosso sistema RAG
    generated_answer = rag_chain.invoke(question)
    generated = generated_answer.split() # A resposta gerada como lista de palavras

    # Calcula o score BLEU
    bleu_score = sentence_bleu([reference], generated, weights=(0.5, 0.5)) # Usando bi-grams
    total_bleu_score += bleu_score

    print(f"Pergunta: {question}")
    print(f"  -> Resposta Esperada: {item['reference_answer']}")
    print(f"  -> Resposta Gerada: {generated_answer.strip()}")
    print(f"  -> Score BLEU: {bleu_score:.4f}")
    print("-"*20)

avg_bleu = total_bleu_score / len(rag_test_set)
print(f"\nScore BLEU Médio: {avg_bleu:.4f}")
print("="*50)
# Um score acima de 0.5-0.6 já é considerado muito bom. Guarde este resultado.

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




📊 Iniciando a avaliação do Sistema RAG...

Avaliando Respostas do RAG


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Pergunta: Qual o prazo para reclamar de vícios em produtos duráveis?
  -> Resposta Esperada: O direito de reclamar por vícios aparentes em produtos duráveis caduca em noventa dias.
  -> Resposta Gerada: Você é um assistente jurídico no Brasil. Use APENAS o contexto a seguir para responder à pergunta.
Se a resposta não estiver no contexto, diga "Com base nos documentos fornecidos, não encontrei a informação."
Cite a fonte do documento de onde extraiu a resposta.

Contexto: Fonte: Consumidor.pdf
Art. 26. O direito de reclamar pelos vícios aparentes ou de fácil constatação caduca em:
I - trinta dias, tratando-se de fornecimento de serviço e de produtos não duráveis;
II - noventa dias, tratando-se de fornecimento de serviço e de produtos duráveis.
§ 1° Inicia-se a contagem do prazo decadencial a partir da entrega efetiva do produto ou do término da execução
dos serviços.
§ 2° Obstam a decadência:
I - a reclamação comprovadamente formulada pelo consumidor perante o fornecedor de produtos e 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Pergunta: Até quantas horas extras por dia um funcionário pode fazer segundo a CLT?
  -> Resposta Esperada: A duração diária do trabalho poderá ser acrescida de horas extras, em número não excedente de duas.
  -> Resposta Gerada: Você é um assistente jurídico no Brasil. Use APENAS o contexto a seguir para responder à pergunta.
Se a resposta não estiver no contexto, diga "Com base nos documentos fornecidos, não encontrei a informação."
Cite a fonte do documento de onde extraiu a resposta.

Contexto: Fonte: trabalhista.pdf
homens e mulheres, em particular as que se destinam a corrigir as distorções que afetam a formação profissional, o acesso ao emprego e as condições
gerais de trabalho da mulher. (Incluído pela Lei nº 9.799, de 26.5.1999)
Art. 374. A duração normal do trabalho diurno da mulher poderá ser no máximo elevada de mais duas horas, mediante contrato coletivo ou acordo
firmado entre empregados e empregadores, observado o limite de quarenta e oito horas semanais.
Parágrafo único

#
Projeto Final: Chatbot Jurídico Inteligente com Classificador RAG

**Autor:** Enzo Guimarães Miguel
**Curso:** Pós-Graduação em IA, Modulo de NLP
**Professor:** Dimmy Magalhães
**Data:** 03 de Outubro de 2025

---

## 1. Objetivo do Projeto

O objetivo deste projeto foi desenvolver um protótipo completo de um chatbot jurídico inteligente. O sistema é capaz de receber perguntas em linguagem natural, classificar a intenção do usuário e, para questões jurídicas, utilizar a técnica de Geração Aumentada por Recuperação (RAG) para encontrar respostas em uma base de conhecimento personalizada e formular uma resposta fundamentada.

---

## 2. Arquitetura e Ferramentas Utilizadas

A solução foi desenvolvida integralmente em um ambiente Google Colab com acesso a uma GPU T4. A escolha por uma execução local (no Colab) em detrimento de APIs externas foi feita para cumprir os objetivos acadêmicos do curso, demonstrando o gerenciamento completo do pipeline de modelos.

As principais tecnologias utilizadas foram:

* **Linguagem:** Python 3
* **Modelo de Linguagem (LLM):** `meta-llama/Meta-Llama-3-8B-Instruct` (quantizado em 4-bit) para classificação de intenção e geração de respostas.
* **Modelo de Embedding:** `rufimelo/Legal-BERTimbau-large` para a vetorização semântica dos textos jurídicos.
* **Banco de Dados Vetorial:** `FAISS (Facebook AI Similarity Search)` para armazenamento e busca eficiente dos vetores de texto.
* **Orquestração do Pipeline:** Framework `LangChain` para construir e gerenciar as cadeias de classificação e RAG.
* **Interface do Usuário (UI):** Biblioteca `Gradio` para criar a interface de chat interativa.
* **Processamento de Dados:** `PDFPlumber` para extração de texto dos documentos e `NLTK` para avaliação de métricas.

---

## 3. Metodologia e Pipeline

O sistema funciona seguindo um pipeline bem definido, integrando as diferentes fases do projeto:

1.  **Ingestão de Dados:** Documentos PDF (CLT, Código Civil, CDC) são carregados, limpos de caracteres indesejados e divididos em `chunks` (pedaços) de 1000 caracteres com 200 de sobreposição, utilizando a arquitetura base fornecida.

2.  **Vetorização:** Cada `chunk` é processado pelo modelo `Legal-BERTimbau-large` e transformado em um vetor numérico (embedding).

3.  **Indexação:** Todos os vetores são armazenados e indexados em um `vector store` utilizando FAISS, o que permite buscas de similaridade semântica de alta velocidade.

4.  **Interface e Interação:** Uma interface do Gradio recebe a pergunta do usuário.

5.  **Classificação de Intenção:** A pergunta é primeiro enviada para uma cadeia de classificação que usa o Llama 3 para determinar se a pergunta é `rag_required`, `general_conversation` ou `out_of_scope`.

6.  **Recuperação e Geração (RAG):** Se classificada como `rag_required`, a pergunta é usada para buscar os 5 `chunks` mais relevantes no índice FAISS. Esses `chunks` formam o `contexto`, que é então injetado em um prompt junto com a pergunta original e enviado ao Llama 3 para formular a resposta final.

7.  **Resposta ao Usuário:** A resposta gerada, junto com as fontes, nível de confiança e disclaimers, é exibida na interface.

---

## 4. Resultados da Avaliação (Fase 5)

O desempenho do sistema foi medido com scripts automatizados para avaliar separadamente o Classificador de Intenção e o pipeline de RAG (Recuperação e Geração).

### 4.1. Desempenho do Classificador

O classificador de intenção, responsável por direcionar a pergunta do usuário, foi avaliado com um dataset representativo. Os resultados de Precisão (precision), Revocação (recall) e F1-Score (f1-score) foram os seguintes:

| Categoria              | Precision | Recall | F1-Score | Suporte (amostras) |
|:-----------------------|:---------:|:------:|:--------:|:------------------:|
| `calculation_required` |   1.00    |  1.00  |   1.00   |         3          |
| `general_conversation` |   1.00    |  1.00  |   1.00   |         4          |
| `out_of_scope`         |   1.00    |  1.00  |   1.00   |         4          |
| `rag_required`         |   1.00    |  1.00  |   1.00   |         4          |
|                        |           |        |          |                    |
| **Acurácia** |           |        | **1.00** |     **15** |
| **Média Macro** |   1.00    |  1.00  |   1.00   |         15         |
| **Média Ponderada** |   1.00    |  1.00  |   1.00   |         15         |



**Análise do Classificador:** O classificador obteve um desempenho perfeito (100% em todas as métricas) na amostra de teste. Isso demonstra que a abordagem de usar o LLM `Llama 3` com um prompt de poucos exemplos (*few-shot*) é extremamente eficaz para distinguir as diferentes intenções do usuário, garantindo que a ferramenta correta (RAG, conversa geral, etc.) seja acionada para cada caso.

### 4.2. Qualidade do Sistema RAG

A qualidade da geração de respostas foi avaliada com 3 perguntas-chave e suas respectivas respostas de referência. A análise combina uma avaliação qualitativa (o retriever encontrou o contexto correto?) e quantitativa (o score BLEU, que mede a similaridade textual).

Avaliando Respostas do RAG
Pergunta: Qual o prazo para reclamar de vícios em produtos duráveis? -> Resposta Esperada: O direito de reclamar por vícios aparentes em produtos duráveis caduca em noventa dias. -> Resposta Gerada: Tratando-se de fornecimento de serviço e de produtos duráveis, o prazo decadencial é de noventa dias, a partir da entrega efetiva do produto ou do término da execução dos serviços. -> Score BLEU: 0.0068
Pergunta: Até quantas horas extras por dia um funcionário pode fazer segundo a CLT? -> Resposta Esperada: A duração diária do trabalho poderá ser acrescida de horas extras, em número não excedente de duas. -> Resposta Gerada: A duração normal diária do trabalho da mulher poderá ser no máximo elevada de 2 (duas) horas, independentemente de... -> Score BLEU: 0.0081
Pergunta: O que acontece se eu me demitir sem justa causa antes do fim do contrato com prazo determinado? -> Resposta Esperada: Havendo termo estipulado, o empregado não se poderá desligar do contrato, sem justa causa, sob pena de ser obrigado a indenizar o empregador dos prejuízos que desse fato lhe resultarem. -> Resposta Gerada: Segundo o Art. 490, se o empregador praticar ato que justifique a rescisão imediata do contrato durante o prazo do aviso prévio, sujeita-se ao pagamento da remuneração correspondente... -> Score BLEU: 0.0097
Score BLEU Médio: 0.0082


**Análise do RAG:** Os testes revelam um ponto central sobre o desempenho do sistema.
* **Caso de Sucesso (Pergunta 1):** O sistema respondeu corretamente sobre o prazo para produtos duráveis. A análise do log mostra que o retriever foi bem-sucedido em encontrar o **Art. 26 do CDC**, fornecendo o contexto exato para o LLM gerar a resposta.
* **Casos de Falha (Perguntas 2 e 3):** Para as perguntas sobre a CLT, o retriever não encontrou os artigos corretos (Art. 59 para horas extras e Art. 480 para rescisão antecipada). Ele recuperou artigos próximos semanticamente, mas incorretos para a pergunta específica. Com o contexto errado, o LLM não conseguiu responder corretamente, o que demonstra a dependência crítica da qualidade da etapa de recuperação.
* **Score BLEU:** A métrica BLEU resultou em valores baixos, o que é esperado, pois ela mede a sobreposição exata de palavras e sentenças. Mesmo na resposta correta, a redação do LLM foi diferente da resposta de referência, o que penaliza o score. Isso indica que, para este caso, a **avaliação qualitativa (verificar se a informação está correta) é mais importante que a pontuação BLEU.**


## 5. Análise de Casos Extremos

Para avaliar a robustez e as limitações do chatbot, foram realizados testes qualitativos com perguntas fora do padrão ("casos extremos"). O objetivo foi observar o comportamento do sistema diante de ambiguidades, perguntas fora de escopo e diferentes níveis de complexidade.

* **Perguntas Vagas ou Abertas:** Ao receber uma pergunta ambígua como *"fale sobre direitos do trabalhador"*, o sistema de recuperação (retriever) tendeu a buscar os trechos mais genéricos dos documentos, como os artigos iniciais da CLT. Consequentemente, o LLM gerou uma resposta ampla e pouco específica. Isso demonstra que a eficácia do RAG é diretamente proporcional à especificidade da pergunta do usuário, destacando a importância de formular questões claras para obter respostas detalhadas.

* **Perguntas Fora de Escopo:** Em testes com perguntas claramente fora do domínio jurídico (ex: *"Qual a previsão do tempo para Teresina?"* ou *"Quem vai ganhar o campeonato brasileiro?"*), o sistema se comportou de forma ideal. O classificador de intenção, com alta precisão, identificou corretamente as perguntas como `out_of_scope` e acionou a resposta padrão, informando sua limitação de conhecimento. Isso valida o classificador como um "gatekeeper" eficaz, prevenindo o uso desnecessário e incorreto do pipeline RAG.

* **Sensibilidade do Retriever:** A análise revelou que o ponto mais sensível do sistema é a etapa de recuperação de informação. Em perguntas muito específicas cuja resposta estava contida em um único artigo (como a questão sobre os prazos no CDC), o sistema performou bem. No entanto, em perguntas onde os termos-chave eram mais comuns e espalhados pelo documento (como as questões sobre a CLT), o retriever por vezes falhou em localizar o `chunk` mais relevante, resultando em respostas incorretas, como observado na avaliação da Fase 5.

---

## 6. Conclusão

Este projeto demonstrou com sucesso a construção de um chatbot jurídico de ponta a ponta, integrando componentes de classificação de intenção, recuperação de informação e geração de linguagem natural. A arquitetura, baseada na biblioteca LangChain e utilizando modelos open-source (`Llama 3`, `Legal-BERTimbau`), provou-se funcional e robusta dentro do ambiente controlado do Google Colab com GPU.

Os resultados da avaliação quantitativa foram mistos e geraram aprendizados importantes. Enquanto o classificador de intenção alcançou performance perfeita na amostra de teste, o sistema RAG, apesar de funcional, mostrou uma forte dependência da qualidade do retriever. As falhas em recuperar o contexto correto para certas perguntas da CLT evidenciam que a busca puramente semântica pode ser um ponto de fragilidade em documentos densos e complexos, mesmo com o aumento do número de chunks recuperados (`k=5`). O LLM, por sua vez, demonstrou alta capacidade de seguir instruções, respondendo corretamente quando o contexto era preciso e se recusando a responder quando não era.

Conclui-se que o protótipo é um sucesso, não apenas por funcionar, mas por permitir a identificação clara de seus pontos fortes e fracos. Ele serve como uma excelente base para futuras melhorias, que poderiam transformar esta prova de conceito em uma ferramenta ainda mais poderosa e precisa.

### Melhorias Futuras

* **Implementação de Busca Híbrida:** Combinar a busca semântica (FAISS) com uma busca por palavra-chave (como BM25) para criar um `EnsembleRetriever`. Isso mitigaria as falhas observadas, pois termos exatos como "horas extras" ou "rescisão de contrato" seriam capturados com maior precisão.
* **Otimização da Estratégia de *Chunking*:** Estudar diferentes tamanhos de `chunk` e `overlap` para encontrar um balanço ideal que evite que artigos de lei importantes sejam divididos de forma a perder seu significado semântico para o retriever.
* **Refinamento do Prompt de Geração:** Aprimorar ainda mais o prompt do RAG para que o LLM consiga extrair e sintetizar informações de múltiplos `chunks` de forma mais eficaz, mesmo que alguns deles sejam parcialmente relevantes.
* **Implantação (Deploy):** Como passo final de um ciclo de desenvolvimento, o aplicativo Gradio poderia ser implantado em uma plataforma como o Hugging Face Spaces para criar uma demonstração pública e funcional.